In [30]:
#objective:
#to see if roulette croupier has muscle memory when spinning the ball

#Todo:
#gather data

#process data
#determine if the ball is within expected half of roulette wheel
    #determine what is correct half
        #determine EXPECTED DISTANCE
        #determine EXPECTED RESULT
        #determine CORRECT_HALF? (Boolean)


#list of correct guess

#Othe Notes:
    #take into account roulette croupier spin clockwise (CW) and anticlockwise (CW) in alternate.
        #to take reference from previous same direction spin
    #margin of error is half of the wheel
    #to determine if 

In [31]:
#import libraries
import pandas as pd
import numpy as np

In [32]:
#import data
df1 = pd.read_excel('Raw Data\Dataset1.xlsx')
df1.head(10)

,SN,RESULT,BALL_DIRECTION,DEALER_GENDER
0,1,19,CW,MALE
1,2,15,ACW,MALE
2,3,21,CW,MALE
3,4,8,ACW,MALE
4,5,21,CW,MALE
5,6,22,ACW,MALE
6,7,4,CW,MALE
7,8,25,ACW,MALE
8,9,23,CW,MALE
9,10,0,ACW,MALE


In [33]:
Roulette_Wheel = [0,32,15,19,4,21,2,25,17,34,6,27,13,36,11,30,8,23,10,5,24,16,33,1,20,14,31,9,22,18,29,7,28,12,35,3,26]
# Roulette_Wheel_Extended = Roulette_Wheel + Roulette_Wheel + Roulette_Wheel
Roulette_Wheel_Extended2 =  Roulette_Wheel + Roulette_Wheel
Roulette_Wheel_Extended2_Reverse = list(reversed(Roulette_Wheel_Extended2))
# print(*Roulette_Wheel_Extended2_Reverse, sep = ", ")

In [34]:
# print(len(Roulette_Wheel))
# print(len(Roulette_Wheel_Extended))

In [35]:
# #determine Number away from previous result (e.g. 19 to 21, CW)
#     #2nd occurance difference from 1st and 3rd occurance, take the minimum > apply clockwise / anticlockwise
# number_to_1st_occurance = Roulette_Wheel_Extended.index(26,0,37)
# number_to_2nd_occurance = Roulette_Wheel_Extended.index(26,37,74)
# number_to_3rd_occurance = Roulette_Wheel_Extended.index(26,74,111)
# print(number_to_1st_occurance)
# print(number_to_2nd_occurance)
# print(number_to_3rd_occurance)

#         #find result to index
# def func_number_to_1st_occurance(result):
#     tmp = Roulette_Wheel_Extended.index(result,0,37)
#     print('func_number_to_1st_occurance'+' Result: '+str(result)+' Index: '+str(tmp))
#     return tmp

# def func_number_to_2nd_occurance(result):
#     tmp = Roulette_Wheel_Extended.index(result,37,74)
#     print('func_number_to_2nd_occurance'+' Result: '+str(result)+' Index: '+str(tmp))
#     return tmp

# def func_number_to_3rd_occurance(result):
#     tmp = Roulette_Wheel_Extended.index(result,74,111)
#     print('func_number_to_3rd_occurance'+' Result: '+str(result)+' Index: '+str(tmp))
#     return tmp


In [36]:
# # Determine Expected Distance
# def func_expected_distance(ppr,pr,dr):
#         #e.g. previous previous cw result is 32, previous cw result is 34, expected distance is 8
#         #ppr means previous previous result, pr means previous result
#         previous_previous_result = ppr
#         previous_result = pr
#         direction = dr

#         #index of previous previous number
#         index_of_previous_previous_result = func_number_to_2nd_occurance(previous_previous_result)
#         print('index of ppr : '+str(index_of_previous_previous_result))

#         #index of previous number
#         index_of_previous_result_1st_occurance = func_number_to_1st_occurance(previous_result)
#         index_of_previous_result_3rd_occurance = func_number_to_3rd_occurance(previous_result)

#         #determine shortest distance (the larger the difference between 1st/3rd occurance to 2nd occurance (previous_previous_result), is the shorter distance in the actual roulette wheel)
#         expected_distance_ppr_to_pr_1st_occurance = abs(index_of_previous_previous_result - index_of_previous_result_1st_occurance)
#         print('expected_distance_ppr_to_pr_1st_occurance: '+str(expected_distance_ppr_to_pr_1st_occurance))
#         expected_distance_ppr_to_pr_3rd_occurance = abs(index_of_previous_previous_result - index_of_previous_result_3rd_occurance)
#         print('expected_distance_ppr_to_pr_2nd_occurance: '+str(expected_distance_ppr_to_pr_3rd_occurance))

#         if expected_distance_ppr_to_pr_1st_occurance <=18:
#                 expected_distance = expected_distance_ppr_to_pr_1st_occurance
#         elif expected_distance_ppr_to_pr_3rd_occurance <= 18:
#                 expected_distance = expected_distance_ppr_to_pr_3rd_occurance
#         else:
#                 expected_distance = max(expected_distance_ppr_to_pr_1st_occurance,expected_distance_ppr_to_pr_3rd_occurance) - 37
                
#         print('expected_distance : '+str(expected_distance))
#         return expected_distance

In [37]:
# func_expected_distance(17,25)

In [38]:

#Function to determine expected distance
def func_expected_distance2(ppr,pr,dr):
        #ppr means previous previous result, pr means previous result, dr means direction
        if dr == 'CW':
                ppr_index = Roulette_Wheel_Extended2.index(ppr,0,37)
                # print(ppr_index)
                pr_index = Roulette_Wheel_Extended2.index(pr,ppr_index,74)
                # print(pr_index)
                expected_distance = pr_index - ppr_index 
                if expected_distance <= 18: 
                       return expected_distance
                else:
                        return expected_distance - 37
        elif dr == 'ACW':
                ppr_index = Roulette_Wheel_Extended2_Reverse.index(ppr,0,37)
                # print(ppr_index)
                pr_index = Roulette_Wheel_Extended2_Reverse.index(pr,ppr_index,74)
                # print(pr_index)
                expected_distance = pr_index - ppr_index 
                if expected_distance <= 18: 
                       return expected_distance
                else:
                        return expected_distance - 37

               
        # #index of previous previous number
        # index_of_previous_previous_result = func_number_to_2nd_occurance(previous_previous_result)
        # print('index of ppr : '+str(index_of_previous_previous_result))

        # #index of previous number
        # index_of_previous_result_1st_occurance = func_number_to_1st_occurance(previous_result)
        # index_of_previous_result_3rd_occurance = func_number_to_3rd_occurance(previous_result)

        # #determine shortest distance (the larger the difference between 1st/3rd occurance to 2nd occurance (previous_previous_result), is the shorter distance in the actual roulette wheel)
        # expected_distance_ppr_to_pr_1st_occurance = abs(index_of_previous_previous_result - index_of_previous_result_1st_occurance)
        # print('expected_distance_ppr_to_pr_1st_occurance: '+str(expected_distance_ppr_to_pr_1st_occurance))
        # expected_distance_ppr_to_pr_3rd_occurance = abs(index_of_previous_previous_result - index_of_previous_result_3rd_occurance)
        # print('expected_distance_ppr_to_pr_2nd_occurance: '+str(expected_distance_ppr_to_pr_3rd_occurance))

        # if expected_distance_ppr_to_pr_1st_occurance <=18:
        #         expected_distance = expected_distance_ppr_to_pr_1st_occurance
        # elif expected_distance_ppr_to_pr_3rd_occurance <= 18:
        #         expected_distance = expected_distance_ppr_to_pr_3rd_occurance
        # else:
        #         expected_distance = max(expected_distance_ppr_to_pr_1st_occurance,expected_distance_ppr_to_pr_3rd_occurance) - 37
                
        # print('expected_distance : '+str(expected_distance))
        # return expected_distance


In [39]:
#test func ed 2
func_expected_distance2(19,21,'ACW')

-2

In [40]:
#function to calculate expected result (based on PR and ED)
def func_expected_result(Previous_Result,Expected_Distance,Ball_Direction):
    pr = Previous_Result
    ed = Expected_Distance
    dir = Ball_Direction
    pr_index = Roulette_Wheel_Extended2.index(pr,0,37)
    #if ball direction is clockwise
    if dir == 'CW':
        #if positive expected distance
        if ed >= 0:
            expected_result_index = Roulette_Wheel_Extended2.index(pr,0,37)+ed
            expected_result = Roulette_Wheel_Extended2[expected_result_index]
            print(expected_result)
            return expected_result
        
        #if negative expected distance
        else: 
            expected_result_index = Roulette_Wheel_Extended2.index(pr,37,74)+ed
            expected_result = Roulette_Wheel_Extended2[expected_result_index]
            print(expected_result)
            return expected_result
    #if ball direction is anti-clockwise
    elif dir == 'ACW':
        #if positive expected distance
        if ed >= 0:
            expected_result_index = Roulette_Wheel_Extended2_Reverse.index(pr,0,37)+ed
            expected_result = Roulette_Wheel_Extended2_Reverse[expected_result_index]
            print(expected_result)
            return expected_result
        #if negative expected distance
        else: 
            expected_result_index = Roulette_Wheel_Extended2_Reverse.index(pr,37,74)+ed
            expected_result = Roulette_Wheel_Extended2_Reverse[expected_result_index]
            print(expected_result)
            return expected_result
    else:
        return 'Error in BALL_DIRECTION'



In [41]:
#test func_expected_result
ER = func_expected_result(32,-7,'ACW')
print(ER)

17
17


In [42]:
#function calculate expected result from df
def func_df_add_expected_result(df):
    #add ppr
    for i in range(4,len(df)):
        df.loc[i,'PPR'] = int(df.loc[i-4,'RESULT'])

    #add pr
    for i in range(4,len(df)):
        df.loc[i,'PR'] = int(df.loc[i-2,'RESULT'])

    #remove unnecessary rows
    df = df.loc[4:len(df)].copy() #copy to avoid chain assignment warning
    df = df.reset_index(drop=True) #to reset index after slicing
    
    #add expected distance to dataframe
    for i in range(0,len(df)):
        # print(df.loc[i,'PPR'])
        # print(df.loc[i,'PR'])
        # print(df.loc[i,'BALL_DIRECTION'])
        df.loc[i,'ED'] = func_expected_distance2(df.loc[i,'PPR'],df.loc[i,'PR'],df.loc[i,'BALL_DIRECTION'])


    # change data type
    df['PPR'] = df['PPR'].astype(int)
    df['PR'] = df['PR'].astype(int)
    df['ED'] = df['ED'].astype(int)

    return df

In [43]:
df1_processed = func_df_add_expected_result(df1)
df1_processed.head(8)

,SN,RESULT,BALL_DIRECTION,DEALER_GENDER,PPR,PR,ED
0,5,21,CW,MALE,19,21,2
1,6,22,ACW,MALE,15,8,-14
2,7,4,CW,MALE,21,21,0
3,8,25,ACW,MALE,8,22,-12
4,9,23,CW,MALE,21,4,-1
5,10,0,ACW,MALE,22,25,-16
6,11,24,CW,MALE,4,23,13
7,12,14,ACW,MALE,25,0,7


In [44]:
df1_processed.dtypes

SN                 int64
RESULT             int64
BALL_DIRECTION    object
DEALER_GENDER     object
PPR                int32
PR                 int32
ED                 int32
dtype: object

In [45]:
    #add expected result to dataframe
def func_add_expected_result_to_df(df):
        #add ER
        for i in range(0,len(df)):
            # print(i)
            df.loc[i,'ER'] = func_expected_result(df.loc[i,'PR'],df.loc[i,'ED'],df.loc[i,'BALL_DIRECTION'])

        #cast
        df['ER'] = df['ER'].astype(int)

        return df


In [46]:
df1_processed = func_add_expected_result_to_df(df1_processed)
df1_processed.head()

25
29
21
19
19
1
29
29
1
36
20
31
22
23
11
27


,SN,RESULT,BALL_DIRECTION,DEALER_GENDER,PPR,PR,ED,ER
0,5,21,CW,MALE,19,21,2,25
1,6,22,ACW,MALE,15,8,-14,29
2,7,4,CW,MALE,21,21,0,21
3,8,25,ACW,MALE,8,22,-12,19
4,9,23,CW,MALE,21,4,-1,19
